In [1]:
import pandas as pd

df = pd.read_csv("[doge] processed_tweets.csv", engine="python")
df_p = pd.read_csv("prices/doge_prices_15m.csv")


In [2]:
df = df.sort_values('Datetime')

In [3]:
from datetime import datetime

i = 0
while i < len(df):
    dt_object = datetime.fromtimestamp(int(float(df.iloc[i]["Datetime"])))
    if dt_object.minute % 15 == 0:
        break
    i += 1


In [4]:
begin = datetime.fromtimestamp(int(float(df.iloc[i]["Datetime"])))
end = datetime.fromisoformat(df_p.iloc[-1]["dateTime"].split()[0])

In [5]:
begin

datetime.datetime(2021, 2, 1, 5, 30)

In [6]:
x = 15
tlow = int(float(df.iloc[i]["Datetime"]))
thigh = tlow + x * 60
tf = end.timestamp()
tlow_array = []
thigh_array = []
score = []


In [7]:
sent, c = 0, 0
while thigh < tf:
    if int(float(df.iloc[i]["Datetime"])) >= tlow and int(float(df.iloc[i]["Datetime"])) < thigh:
        sent += df.iloc[i]["polarity"]
        c += 1
        i += 1
    else:
        tlow_array.append(tlow)
        thigh_array.append(thigh)
        score.append(sent)
        tlow = thigh
        thigh = tlow + x * 60
        sent = 0
        c = 0


In [8]:
df1 = pd.DataFrame()
df1["time start"] = tlow_array
df1["time high"] = thigh_array
df1["sentiment"] = score


In [9]:
df1

,time start,time high,sentiment
0,1612137600,1612138500,21.574494
1,1612138500,1612139400,26.129070
2,1612139400,1612140300,29.471794
3,1612140300,1612141200,25.015827
4,1612141200,1612142100,20.294897
...,...,...,...
2660,1614531600,1614532500,38.047072
2661,1614532500,1614533400,23.071823
2662,1614533400,1614534300,26.315925
2663,1614534300,1614535200,35.276263


In [10]:
import numpy as np
senti = np.array(df1["sentiment"])

In [11]:
senti

array([21.57449352, 26.12907041, 29.47179435, ..., 26.31592491,
       35.27626321, 36.12431589])

In [12]:
i = 0
begin_val = str(begin).replace(":", "-")
while df_p.iloc[i].dateTime < str(begin_val):
    i += 1
df_p = df_p[i:]

In [13]:
df_p.insert(6,"Sentiment",senti)

In [14]:
df_p

,dateTime,open,high,low,close,volume,Sentiment
22,2021-02-01 05-30-00,0.040052,0.040557,0.039120,0.040317,225333094.0,21.574494
23,2021-02-01 05-45-00,0.040335,0.040904,0.039804,0.040685,188514797.0,26.129070
24,2021-02-01 06-00-00,0.040685,0.042000,0.040506,0.040840,399989498.0,29.471794
25,2021-02-01 06-15-00,0.040801,0.041221,0.040368,0.041182,212693163.0,25.015827
26,2021-02-01 06-30-00,0.041194,0.041220,0.038444,0.038866,383893056.0,20.294897
...,...,...,...,...,...,...,...
2682,2021-02-28 23-00-00,0.048655,0.049170,0.048655,0.048959,18585671.0,38.047072
2683,2021-02-28 23-15-00,0.048958,0.049110,0.048802,0.048818,7464841.0,23.071823
2684,2021-02-28 23-30-00,0.048818,0.049004,0.048401,0.048467,12308931.0,26.315925
2685,2021-02-28 23-45-00,0.048439,0.048453,0.047900,0.048158,27693957.0,35.276263


In [15]:
df_p.to_csv("aligned_doge.csv")